In [36]:
# Do this from terminal first pls:
# $ jupyter nbextension enable --py --sys-prefix widgetsnbextension
# $ jupyter nbextension enable --py --sys-prefix gmaps
import gmaps
import gmaps.datasets
import pandas as pd

In [46]:
api_key = 'your_api_key_here'

In [47]:
# dataset can be downloaded here https://www.kaggle.com/imdevskp/corona-virus-report
df = pd.read_csv('./corona/covid_19_clean_complete.csv')

In [48]:
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.8257,117.2264,1/22/20,1.0,0.0,0.0
1,Beijing,Mainland China,40.1824,116.4142,1/22/20,14.0,0.0,0.0
2,Chongqing,Mainland China,30.0572,107.8740,1/22/20,6.0,0.0,0.0
3,Fujian,Mainland China,26.0789,117.9874,1/22/20,1.0,0.0,0.0
4,Gansu,Mainland China,36.0611,103.8343,1/22/20,0.0,0.0,0.0


In [49]:
df = df[ df['Country/Region'] == 'US' ]

In [50]:
df['Date'] = pd.to_datetime(df['Date'])
df['Week'] = df['Date'].dt.week

In [51]:
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Week
35,"King County, WA",US,47.6062,-122.3321,2020-01-22,1.0,0.0,0.0,4
36,"Cook County, IL",US,41.7377,-87.6976,2020-01-22,0.0,0.0,0.0,4
37,"Tempe, AZ",US,33.4255,-111.9400,2020-01-22,0.0,0.0,0.0,4
47,"Los Angeles, CA",US,34.0522,-118.2437,2020-01-22,0.0,0.0,0.0,4
63,"Santa Clara, CA",US,37.3541,-121.9552,2020-01-22,0.0,0.0,0.0,4


In [52]:
df.groupby('Week').size()

Week
4     315
5     441
6     441
7     441
8     441
9     441
10    315
dtype: int64

In [53]:
315 + 441

756

In [54]:
df.columns = ['state', 'country', 'lat', 'long', 'date', 'confirmed', 'deaths', 'recovered', 'week']

In [55]:
df.head()

,state,country,lat,long,date,confirmed,deaths,recovered,week
35,"King County, WA",US,47.6062,-122.3321,2020-01-22,1.0,0.0,0.0,4
36,"Cook County, IL",US,41.7377,-87.6976,2020-01-22,0.0,0.0,0.0,4
37,"Tempe, AZ",US,33.4255,-111.9400,2020-01-22,0.0,0.0,0.0,4
47,"Los Angeles, CA",US,34.0522,-118.2437,2020-01-22,0.0,0.0,0.0,4
63,"Santa Clara, CA",US,37.3541,-121.9552,2020-01-22,0.0,0.0,0.0,4


In [56]:
from IPython.display import display
import ipywidgets as widgets

import gmaps
import gmaps.datasets

gmaps.configure(api_key=api_key)


class CovidExplorer(object):
    """
    Jupyter widget for exploring the COVID-19 dataset.

    The user uses the slider to choose a week. This renders
    a heatmap of infections up to that week.
    """

    def __init__(self, df):
        self._df = df
        self._heatmap = None
        self._slider = None
        initial_week = min(self._df['week'])

        title_widget = widgets.HTML(
            '<h3>US COVID-19 Infections, by week</h3>'
            '<h4>Data from <a href="https://www.kaggle.com/imdevskp/corona-virus-report">Kaggle</a></h4>'
        )

        map_figure = self._render_map(initial_week)
        controls = self._render_controls(initial_week)
        self._container = widgets.VBox([title_widget, controls, map_figure])

    def render(self):
        display(self._container)

    def _on_week_change(self, change):
        week = self._slider.value
        self._heatmap.locations = self._locations_for_week(week)
        self._infected.value = self._total_infected_text_for_week(week)
        self._recovered.value = self._total_recovered_text_for_week(week)
        self._deaths.value = self._total_deaths_text_for_week(week)
        return self._container

    def _render_map(self, initial_week):
        fig = gmaps.figure(map_type='HYBRID')
        self._heatmap = gmaps.heatmap_layer(
            self._locations_for_week(initial_week),
            max_intensity=100,
            point_radius=8
        )
        fig.add_layer(self._heatmap)
        return fig

    def _render_controls(self, initial_week):
        self._slider = widgets.IntSlider(
            value=initial_week,
            min=min(self._df['week']),
            max=max(self._df['week']),
            description='Week',
            continuous_update=False
        )
        self._infected = widgets.Label(
            value=self._total_infected_text_for_week(initial_week)
        )
        
        self._recovered = widgets.Label(
            value=self._total_recovered_text_for_week(initial_week)
        )
        
        self._deaths = widgets.Label(
            value=self._total_deaths_text_for_week(initial_week)
        )
        
        self._slider.observe(self._on_week_change, names='value')
        controls = widgets.HBox(
            [self._slider, self._infected, self._recovered, self._deaths],
            layout={'justify_content': 'space-between'}
        )
        return controls

    def _locations_for_week(self, week):
        return self._df[self._df['week'] <= week][['lat', 'long']]

    def _total_infected_for_week(self, week):
        return int(self._df[self._df['week'] <= week]['confirmed'].sum())
    
    def _total_recovered_for_week(self, week):
        return int(self._df[self._df['week'] <= week]['recovered'].sum())
    
    def _total_deaths_for_week(self, week):
        return int(self._df[ (self._df['week'] <= week) ]['deaths'].sum())

    def _total_infected_text_for_week(self, week):
        return '{} infected'.format(self._total_infected_for_week(week))
    
    def _total_recovered_text_for_week(self, week):
        return '{} recovered'.format(self._total_recovered_for_week(week))
    
    def _total_deaths_text_for_week(self, week):
        return '{} casualties'.format(self._total_deaths_for_week(week))

    
# df = gmaps.datasets.load_dataset_as_df('acled_africa_by_year')
CovidExplorer(df).render()